# Demand Forecasting Analysis

This notebook demonstrates demand forecasting capabilities using various time series models.

## Key Features:
- Time series data preparation
- Multiple forecasting models (Prophet, Linear Trend, Seasonal Naive)
- Model accuracy comparison
- Business insights generation
- Category-wise forecasting

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sys
import os

# Add src directory to path
sys.path.append('../src')

from forecasting import DemandForecaster

## 1. Load and Prepare Data

In [ ]:
# Load the processed data
df = pd.read_csv('../data/processed_sales_data.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"Data shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Total revenue: ${df['total_amount'].sum():,.2f}")

# Prepare time series data
ts_data = df.groupby('date')['total_amount'].sum().reset_index()
ts_data.columns = ['ds', 'y']

print(f"\nTime series data prepared: {len(ts_data)} days of data")

## 2. Exploratory Time Series Analysis

In [ ]:
# Initialize forecaster
forecaster = DemandForecaster()

# Analyze trends
trend_analysis = forecaster.analyze_trends(ts_data)

print("=== TREND ANALYSIS ===")
for metric, analysis in trend_analysis.items():
    print(f"\n📈 {metric.upper()}:")
    print(f"   Overall Trend: {analysis['trend_direction']}")
    print(f"   Growth Rate: {analysis['growth_rate']:.2f}% per day")
    print(f"   Volatility: {analysis['volatility']:.2f}")
    print(f"   Peak Day: {analysis['peak_date']} (${analysis['peak_value']:,.2f})")
    print(f"   Average: ${analysis['average']:.2f}")

In [ ]:
# Visualize trends
fig = forecaster.visualize_trends(ts_data)
fig.update_layout(
    title='Daily Sales Trends and Patterns',
    height=600
)
fig.show()

## 3. Build and Compare Forecasting Models

In [ ]:
# Compare model accuracy
accuracy_results = forecaster.compare_model_accuracy(ts_data)

print("=== MODEL ACCURACY COMPARISON ===")
accuracy_df = pd.DataFrame(accuracy_results).T
accuracy_df = accuracy_df.round(4)
print(accuracy_df)

# Find best model
best_model = accuracy_df['MAPE'].idxmin()
print(f"\n🏆 Best performing model: {best_model} (MAPE: {accuracy_df.loc[best_model, 'MAPE']:.4f})")

In [ ]:
# Generate forecasts
forecast_results = forecaster.forecast_demand(ts_data, periods=30)

print("=== FORECAST RESULTS ===")
print(f"Forecast period: {forecast_results['forecast_period']} days")
print(f"Model used: {forecast_results['model_used']}")
print(f"Forecast accuracy (MAPE): {forecast_results['accuracy']['MAPE']:.4f}")

# Show forecast summary
forecast_df = forecast_results['forecast']
print(f"\nForecast Summary:")
print(f"Average daily forecast: ${forecast_df['yhat'].mean():,.2f}")
print(f"Total 30-day forecast: ${forecast_df['yhat'].sum():,.2f}")
print(f"Highest forecast day: ${forecast_df['yhat'].max():,.2f}")
print(f"Lowest forecast day: ${forecast_df['yhat'].min():,.2f}")

## 4. Visualize Forecasts

In [ ]:
# Visualize forecast
fig = forecaster.visualize_forecast(forecast_results, ts_data)
fig.update_layout(
    title='30-Day Demand Forecast',
    height=500
)
fig.show()

## 5. Business Insights

In [ ]:
# Generate business insights
insights = forecaster.generate_insights(forecast_results, ts_data)

print("\n=== BUSINESS INSIGHTS ===")
for insight in insights:
    print(f"• {insight}")

## 6. Category-wise Forecasting

In [ ]:
# Category-wise forecasting
print("\n=== CATEGORY-WISE FORECASTING ===")

category_forecasts = {}

for category in df['category'].unique():
    print(f"\nForecasting for {category}...")
    
    # Filter data for category
    category_data = df[df['category'] == category]
    category_daily = category_data.groupby('date')['total_amount'].sum().reset_index()
    category_daily.columns = ['ds', 'y']
    
    if len(category_daily) >= 30:  # Only forecast if enough data
        category_forecast = forecaster.forecast_demand(category_daily, periods=30)
        category_forecasts[category] = category_forecast
        
        # Show category insights
        category_insights = forecaster.generate_insights(category_forecast, category_daily)
        for insight in category_insights[:2]:  # Show top 2 insights
            print(f"  • {insight}")
    else:
        print(f"  • Insufficient data for {category} (only {len(category_daily)} days)")

## 7. Save Results

In [ ]:
# Save forecast results
forecast_results['forecast'].to_csv('../data/demand_forecast.csv', index=False)

# Save category forecasts
if category_forecasts:
    for category, forecast in category_forecasts.items():
        filename = f"../data/forecast_{category.lower().replace(' ', '_')}.csv"
        forecast['forecast'].to_csv(filename, index=False)

print("✅ Forecast results saved to data/ directory")